In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf

%pip install keras-tcn --no-dependencies

%pip install tensorflow-addons[tensorflow-gpu] --no-dependencies

%pip install pandas --upgrade

%pip install multivariate_cwru

%pip install tqdm

%pip install scipy

%pip install sklearn


tf.config.list_physical_devices(device_type=None)
physical_devices = tf.config.list_physical_devices('GPU')
visible_devices = tf.config.get_visible_devices()
print(visible_devices)
num_GPU = len(tf.config.experimental.list_physical_devices('/physical_device:GPU:0'))
#if num_GPU == 1:
#tf.config.experimental.set_memory_growth(physical_devices[:], True)
#elif num_GPU == 2:
#tf.config.experimental.set_memory_growth(physical_devices[1], True)
tf.config.experimental.set_memory_growth(physical_devices[0], True)
#elif num_GPU == 4:
    #tf.config.experimental.set_memory_growth(physical_devices[1], True)
    #tf.config.experimental.set_memory_growth(physical_devices[0], True)
    #tf.config.experimental.set_memory_growth(physical_devices[1], True)
    #tf.config.experimental.set_memory_growth(physical_devices[0], True)
#print("Num GPUs Available: ", num_GPU)
#tf.debugging.set_log_device_placement(False)
#mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:1"])
#strategy = tf.distribute.MirroredStrategy(
    #cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
# Data science libraries
import scipy.io
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics

#Deep Learning pkgs
from tensorflow.keras import backend as K, Input, Model, optimizers
from tensorflow.keras.layers import Dense, Activation
import tensorflow.keras.metrics
from tensorflow.keras.metrics import SparseCategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, ModelCheckpoint, Callback
from tensorflow.keras.activations import swish
K.backend()

# Python
from IPython.core.debugger import set_trace
from pathlib import Path
import os
import datetime
import time
import glob
from timeit import default_timer as timer


#Project Specific
import tcn_ed
from tcn_ed import TCN, tcn_full_summary, compiled_tcn
from help_pre import create_data_batcht as Create_Batch, create_pred_batch
import multivariate_cwru

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [2]:
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [2]:
data1 = multivariate_cwru.CWRU("12DriveEndFault", 1000000, 1, 1,2,'1797',"1797","1772","1750","1730",normal_condition = True)
data2 = multivariate_cwru.CWRU("12FanEndFault", 2400, 1, 1,2,'1797',"1797","1772","1750","1730",normal_condition = True)
data3 = multivariate_cwru.CWRU("48DriveEndFault", 1000000, 1, 1,2,"1797",normal_condition = True)
#data = multivariate_cwru.CWRU("48DriveEndFault", 2400, 0, 1,2,"1772",normal_condition = True) # Throws an error
data4 = multivariate_cwru.CWRU("48DriveEndFault", 1000000, 1, 1,2,"1750",normal_condition = True)
data5 = multivariate_cwru.CWRU("48DriveEndFault", 1000000, 1, 1,2,"1730",normal_condition = True)


In [4]:
os.getcwd()

'/home/jupyter/CWRU_TCN'

In [3]:
working_dir = Path('.')
DATA1_PATH = Path("./Datasets")
DATA_PATH = Path("./Datasets/CWRU")
save_model_path = working_dir / 'Model' 
DE_path = DATA_PATH / '12DriveEndFault'
DE_path1 = DE_path / '1730'
DE_path2 = DE_path / '1750'
DE_path3 = DE_path / '1772'
DE_path4 = DE_path / '1797'

FE_path = DATA_PATH / '12FanEndFault'
FE_path1 = FE_path / '1730'
FE_path2 = FE_path / '1750'
FE_path3 = FE_path / '1772'
FE_path4 = FE_path / '1797'

DE48_path = DATA_PATH / '48DriveEndFault'
DE48_path1 = DE48_path / '1730'
DE48_path2 = DE48_path / '1750'
DE48_path3 = DE48_path / '1772'
DE48_path4 = DE48_path / '1797'

Normal_path = DATA_PATH / 'NormalBaseline'
Normal_path1 = Normal_path / '1730'
Normal_path2 = Normal_path / '1750'
Normal_path3 = Normal_path / '1772'
Normal_path4 = Normal_path / '1797'

#Paths = [DE_path1, DE_path2, DE_path3, DE_path4, FE_path1, FE_path2, FE_path3, FE_path4, DE48_path1, DE48_path2,  DE48_path4, Normal_path1, Normal_path2, Normal_path3, Normal_path4]
Paths = [DE_path1, DE_path2, DE_path3, DE_path4, FE_path1, FE_path2, FE_path3, FE_path4, Normal_path1, Normal_path2, Normal_path3, Normal_path4]
data_path = Paths

val_path = DATA1_PATH / 'for_pred'
Val_path= [val_path]



segment_length = 2400 # The length of the window used to take sub-arrays from each file
step_size = 300 # the amount the window moves along the sub-array
split_perc = 0.2 # percentage of data to put towards the test set
b_size = 1024
b_size1 = int(b_size*2)
#b_size2 = int(b_size / 4)

#step_length = step_size

for path in [DATA_PATH, save_model_path]:
    if not path.exists():
        path.mkdir(parents=True)
        
data_path = Paths

In [4]:
with mirrored_strategy.scope():
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                                  factor=0.3,   
                                  patience=40, 
                                  min_lr=0.000000015,
                                  verbose=1, 
                                  cooldown=5)

    ES_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                             min_delta=0.00001, 
                                             patience=70, 
                                             verbose=1, 
                                             mode='auto', 
                                             baseline=None, 
                                             restore_best_weights=True)



    callback = [reduce_lr, ES_cb]

def adjust_dilations(dilations: list):
    if all([is_power_of_two(i) for i in dilations]):
        return dilations
    else:
        new_dilations = [2 ** i for i in dilations]
        return new_dilations
        
def pred_accuracy(y_true, y_pred):
            # reshape in case it's in shape (num_samples, 1) instead of (num_samples,)
            # convert dense predictions to labels
            y_pred_labels = K.cast(y_pred, K.floatx())
            return K.cast(K.equal(y_true, y_pred_labels), K.floatx())

def create_model(num_feat,
                 num_classes,
                 filt_num,
                 kernel_num,
                 stack, dilation,
                 lr,
                 drop_rate,
                 segment_length,
                 use_skip,
                 opt):

    with mirrored_strategy.scope():

        input_layer = Input(shape=(segment_length, num_feat))

        def get_opt():
            if opt == 'adam':
                return optimizers.Adam(lr=lr)
            elif opt == 'rmsprop':
                return optimizers.RMSprop(lr=lr)
            else:
                raise Exception('Only Adam and RMSProp are available here')
            
        x = TCN(nb_filters=filt_num,
                kernel_size=kernel_num,
                nb_stacks=stack,
                dilations=[2 ** i for i in range(dilation)],
                padding='causal',
                use_skip_connections=use_skip,
                dropout_rate=drop_rate,
                return_sequences=False,
                activation='swish', 
                kernel_initializer='he_uniform',
                use_batch_norm=True,
                use_layer_norm=False,
                use_weight_norm=False,
                name='Model')(input_layer)

        tcn = TCN(nb_filters=filt_num,
                  kernel_size=kernel_num,
                  nb_stacks=stack,
                  dilations=[2 ** i for i in range(dilation)],
                  padding='causal',
                  use_skip_connections=use_skip,
                  dropout_rate=drop_rate,
                  return_sequences=False,
                  activation='swish', 
                  kernel_initializer='he_uniform',
                  use_batch_norm=True,
                  use_layer_norm=False,
                  use_weight_norm=False,
                  name='Model')

        print('Receptive Field Size: %s' % tcn.receptive_field)



        print('x.shape=', x.shape)


        # classification
        x = Dense(num_classes)(x)
        x = Activation('softmax', dtype='float32')(x)
        output_layer = x
        model = Model(input_layer, output_layer)

        # https://github.com/keras-team/keras/pull/11373
        # It's now in Keras@master but still not available with pip.
        # TODO remove later.

    with mirrored_strategy.scope():
        model.compile(get_opt(), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

        print('model.x = {}'.format(input_layer.shape))
        print('model.y = {}'.format(output_layer.shape))

        print('Filter Length: %s' % filt_num)
        print('Kernel Size: %s' % kernel_num)
        print('Dilation: %s' % dilation)
        print('Learning Rate: %s' % lr)
        print('Dropout Rate: %s' % drop_rate)

        model.summary()


        return model

def train_func(train_data,
               test_dataset,
               Val_dataset,
               predict_labels,
               segment_length,
               runs,
               filt_num,
               kernel_num,
               dilation,
               stack,
               learn_r,
               drop_rate,
               Model_num):
    
    
        
    time1 = timer()
        
    working_dir = Path('.')
    model_path = working_dir / 'Model' / ('Model_{}_k{}_s{}_di{}_dr{}_L{}'.format(Model_num, kernel_num, stack, dilation, drop_rate, segment_length))
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                                  factor=0.3,   
                                  patience=40, 
                                  min_lr=0.000015,
                                  verbose=0, 
                                  cooldown=5)

    ES_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                             min_delta=0.00001, 
                                             patience=70, 
                                             verbose=0, 
                                             mode='auto', 
                                             baseline=None, 
                                             restore_best_weights=True)
    

    # to load best weights model.load_weights(latest)

    callback = ES_cb# ] [reduce_lr, 
    #seg_length = None
    time_out = timer()-time1
    print('Time till start of create model %s' % time_out)
    time2 = timer()
    model = create_model(num_feat=1,
                         num_classes=4,
                         filt_num=filt_num,
                         kernel_num=kernel_num,
                         stack=stack,
                         dilation=dilation,
                         lr=learn_r,
                         drop_rate=drop_rate,
                         segment_length = segment_length,
                         use_skip=True,
                         opt='adam')

    time3 = timer()
    time_out = time3-time2
    print('Time to create model %s' % time_out)
    with mirrored_strategy.scope():
        history = model.fit(train_data,
                  epochs=runs,
                  verbose=2,
                  callbacks=callback,
                  validation_data=test_dataset)
    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.show()
    plt.plot(history.history['loss'])
    plt.show()
    time_out = timer()-time3
    print('Time to fit model %s' % time_out)
    with mirrored_strategy.scope():
        y_preds = model.predict(Val_dataset,
                                       verbose=1,
                                       callbacks=callback,
                                      )
        y_argmax2 = np.argmax(y_preds,axis=1)
        predict_labels = np.squeeze(predict_labels[:], axis=1)
        #for pred1, pred2, pred3, pred4 in y_pred:
        #y_argmax = []
        #i=0
        perc_score = tf.dtypes.cast((sum(pred_accuracy(predict_labels, y_argmax2))/(len(predict_labels))), tf.float16)
        Accuracy_test = float(perc_score)
        print('Prediction Accuracy: %s' % Accuracy_test)
        con_mat = metrics.confusion_matrix(predict_labels, y_argmax2)
        class_report = metrics.classification_report(predict_labels, y_argmax2, digits=3)
        print(con_mat, '\n\n')
        print(class_report, '\n\n')
    
    

    
    d = {'segment length':[segment_length], 'filters': [filt_num], 'kernel size': [kernel_num], 'stacks':[stack], 'dropout': [drop_rate], 'lr': [learn_r], 'dilation': [dilation], 'Training Time': time_out, 'train loss': [history.history['loss'][-1]], 'train acc': [history.history['sparse_categorical_accuracy'][-1]], 'eval acc': [history.history['val_sparse_categorical_accuracy'][-1]], 'Eval Acc': [Accuracy_test],'C1 correct': [con_mat[0][0]], 'C1 as C2':[con_mat[0][1]], 'C1 as C3':[con_mat[0][2]], 'C1 as C4':[con_mat[0][3]],'C2 as C1': [con_mat[1][0]], 'C2 correct':[con_mat[1][1]], 'C2 as C3':[con_mat[1][2]], 'C2 as C4':[con_mat[1][3]],'C3 as C1': [con_mat[2][0]], 'C3 as C2':[con_mat[2][1]], 'C3 correct':[con_mat[2][2]], 'C3 as C4':[con_mat[2][3]],'C4 as C1': [con_mat[3][0]], 'C4 as C2':[con_mat[3][1]], 'C4 as C3':[con_mat[3][2]], 'C4 correct':[con_mat[3][3]]}
    
    
    append_list2_in = pd.DataFrame.from_dict(d)
    
    time_out = timer()-time1
    print('Total Time %s' % time_out)
    
    return history, y_preds, append_list2_in

def create_model2(num_feat,
                 num_classes,
                 filt_num,
                 kernel_num,
                 stack, dilation,
                 lr,
                 drop_rate,
                 segment_length,
                 use_skip,
                 opt):




    with mirrored_strategy.scope():
        #x =Sequential()

        
        input_layer = Input(shape=(segment_length, num_feat))

        def get_opt():
            if opt == 'adam':
                return optimizers.Adam(lr=lr)
            elif opt == 'rmsprop':
                return optimizers.RMSprop(lr=lr)
            else:
                raise Exception('Only Adam and RMSProp are available here')
            
        x = TCN(nb_filters=filt_num,
                kernel_size=kernel_num,
                nb_stacks=stack,
                dilations=[2 ** i for i in range(dilation)],
                padding='causal',
                use_skip_connections=use_skip,
                dropout_rate=drop_rate,
                return_sequences=False,
                activation='swish', 
                kernel_initializer='he_uniform',
                use_batch_norm=True,
                use_layer_norm=False,
                use_weight_norm=False,
                name='Model')(input_layer)

        tcn = TCN(nb_filters=filt_num,
                  kernel_size=kernel_num,
                  nb_stacks=stack,
                  dilations=[2 ** i for i in range(dilation)],
                  padding='causal',
                  use_skip_connections=use_skip,
                  dropout_rate=drop_rate,
                  return_sequences=False,
                  activation='swish', 
                  kernel_initializer='he_uniform',
                  use_batch_norm=True,
                  use_layer_norm=False,
                  use_weight_norm=False,
                  name='Model')

        print('Receptive Field Size: %s' % tcn.receptive_field)



        print('x.shape=', x.shape)


        # classification
        x = Dense(num_classes)(x)
        x = Activation('softmax', dtype='float32')(x)
        output_layer = x
        model = Model(input_layer, output_layer)

        # https://github.com/keras-team/keras/pull/11373
        # It's now in Keras@master but still not available with pip.
        # TODO remove later.

    with mirrored_strategy.scope():
        model.compile(get_opt(), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

        print('model.x = {}'.format(input_layer.shape))
        print('model.y = {}'.format(output_layer.shape))

        print('Filter Length: %s' % filt_num)
        print('Kernel Size: %s' % kernel_num)
        print('Dilation: %s' % dilation)
        print('Learning Rate: %s' % lr)
        print('Dropout Rate: %s' % drop_rate)

        model.summary()


        return model

def train_func2(train_data,
               test_dataset,
               Val_dataset,
               predict_labels,
               segment_length,
               runs,
               filt_num,
               kernel_num,
               dilation,
               stack,
               learn_r,
               drop_rate,
               Model_num):
    
    
        
    time1 = timer()
        
    working_dir = Path('.')
    model_path = working_dir / 'Model' / ('Model_{}_k{}_s{}_di{}_dr{}_L{}.H5'.format(Model_num, kernel_num, stack, dilation, drop_rate, segment_length))
    
    

    #seg_length = None
    time_out = timer()-time1
    print('Time till start of create model %s' % time_out)
    time2 = timer()
    model = create_model(num_feat=1,
                         num_classes=4,
                         filt_num=filt_num,
                         kernel_num=kernel_num,
                         stack=stack,
                         dilation=dilation,
                         lr=learn_r,
                         drop_rate=drop_rate,
                         segment_length = segment_length,
                         use_skip=True,
                         opt='adam')

    time3 = timer()
    time_out = time3-time2
    print('Time to create model %s' % time_out)
    with mirrored_strategy.scope():
        history = model.fit(train_data,
                  epochs=runs,
                  verbose=2,
                  callbacks=callback,
                  validation_data=test_dataset)
    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.show()
    plt.plot(history.history['loss'])
    plt.show()
    time_out = timer()-time3
    print('Time to fit model %s' % time_out)
    with mirrored_strategy.scope():
        y_preds = model.predict(Val_dataset,
                                       verbose=0,
                                       callbacks=callback,
                                      )
        y_argmax2 = np.argmax(y_preds,axis=1)
        predict_labels = np.squeeze(predict_labels[:], axis=1)
        #for pred1, pred2, pred3, pred4 in y_pred:
        #y_argmax = []
        #i=0
        perc_score = tf.dtypes.cast((sum(pred_accuracy(predict_labels, y_argmax2))/(len(predict_labels))), tf.float16)
        Accuracy_test = float(perc_score)
        print('Prediction Accuracy: %s' % Accuracy_test)
        con_mat = metrics.confusion_matrix(predict_labels, y_argmax2)
        class_report = metrics.classification_report(predict_labels, y_argmax2, digits=3)
        print(con_mat, '\n\n')
        print(class_report, '\n\n')
    
    

    
    d = {'segment length':[segment_length], 'filters': [filt_num], 'kernel size': [kernel_num], 'stacks':[stack], 'dropout': [drop_rate], 'lr': [learn_r], 'dilation': [dilation], 'Training Time': time_out, 'train loss': [history.history['loss'][-1]], 'train acc': [history.history['sparse_categorical_accuracy'][-1]], 'eval acc': [history.history['val_sparse_categorical_accuracy'][-1]], 'Eval Acc': [Accuracy_test],'C1 correct': [con_mat[0][0]], 'C1 as C2':[con_mat[0][1]], 'C1 as C3':[con_mat[0][2]], 'C1 as C4':[con_mat[0][3]],'C2 as C1': [con_mat[1][0]], 'C2 correct':[con_mat[1][1]], 'C2 as C3':[con_mat[1][2]], 'C2 as C4':[con_mat[1][3]],'C3 as C1': [con_mat[2][0]], 'C3 as C2':[con_mat[2][1]], 'C3 correct':[con_mat[2][2]], 'C3 as C4':[con_mat[2][3]],'C4 as C1': [con_mat[3][0]], 'C4 as C2':[con_mat[3][1]], 'C4 as C3':[con_mat[3][2]], 'C4 correct':[con_mat[3][3]]}
    
    
    append_list2_in = pd.DataFrame.from_dict(d)
       
    time_out = timer()-time1
    print('Total Time %s' % time_out)
    
    return model, history, y_preds, append_list2_in

```'N':0, 'B':1, 'IR':2, 'OR':3```


In [5]:
b_size = 256 #normally 2048 but 512 on colab
b_size2 = int(b_size / 4)
step_size = 300
step_length=step_size
segment_length = 2400
with mirrored_strategy.scope():
    train_dataset, test_dataset, y_s1 = Create_Batch(data_path, split_perc, segment_length, step_length, b_size)
waste_split = 0.1


Sig Divide Time: 1.186478853225708
Sig Divide Time: 4.908093690872192
class balance of train frame: 3    31861
2    15915
1    14333
0     8849
Name: label, dtype: int64
class balance of validation (test) frame: 3    7169
1    4775
2    3185
0    2415
Name: label, dtype: int64


In [7]:
with mirrored_strategy.scope():
    val_dataset, X_v1, predict_labels= create_pred_batch(Val_path=Val_path, segment_length=segment_length, step_length=step_size, b_size=int(b_size/4))

Sig Divide Time: 1.586183786392212


# Batch Norm

In [ ]:
append_list_time1.sort_values(by=['segment length', 'kernel size', 'stacks', 'dilation', 'filters'], ascending=[True, False, False, False, True])
append_list_time1.to_csv(r'{}.csv'.format('List_Check_300'), index = False, header=['segment length', 'filters', 'kernel size', 'stacks', 'dropout', 'lr', 'dilation','Training Time', 'train loss', 'train acc', 'eval acc','Eval Acc','C1 correct', 'C1 as C2', 'C1 as C3', 'C1 as C4','C2 as C1', 'C2 correct', 'C2 as C3', 'C2 as C4','C3 as C1', 'C3 as C2', 'C3 correct', 'C3 as C4','C4 as C1', 'C4 as C2', 'C4 as C3', 'C4 correct'])

In [ ]:
append_list_time1

# Save and Reload Model

In [8]:
# Too big for 16GB

tf.keras.backend.clear_session()
b_size = 2048 #normally 2048 but 512 on colab
b_size2 = int(b_size / 4)
step_size = 300
step_length=step_size
segment_length = 2400
with mirrored_strategy.scope():
    train_dataset, test_dataset, y_s1 = Create_Batch(data_path, split_perc, segment_length, step_length, b_size)
    val_dataset, X_v1, predict_labels= create_pred_batch(Val_path=Val_path, segment_length=segment_length, step_length=step_size, b_size=int(b_size/4))
waste_split = 0.1
append_list_time1 = pd.DataFrame()
with mirrored_strategy.scope():
    
    #Tuned Param dropout
    Learning_Rate = 0.0125
    Drop_rate = 0.175
    Repeats = 75

    m_count = 1
    print('Model Number %s' % m_count)
    history, y_preds, res_list = train_func2(train_data=train_dataset,
                                            test_dataset=test_dataset,
                                            Val_dataset=val_dataset,
                                            predict_labels=predict_labels, 
                                            filt_num=16,
                                            kernel_num=11,
                                            dilation=7,
                                            stack=1,
                                            learn_r=Learning_Rate,
                                            drop_rate=Drop_rate,
                                            runs=Repeats,
                                            Model_num = m_count, 
                                            segment_length = segment_length)
    append_list_time1 = append_list_time1.append(res_list, ignore_index=True)
    Learning_Rate = 0.0125
    Drop_rate = 0.3
    Repeats = 75

    m_count = 2
    print('Model Number %s' % m_count)
    history, y_preds, res_list = train_func2(train_data=train_dataset,
                                            test_dataset=test_dataset,
                                            Val_dataset=val_dataset,
                                            predict_labels=predict_labels, 
                                            filt_num=16,
                                            kernel_num=11,
                                            dilation=7,
                                            stack=1,
                                            learn_r=Learning_Rate,
                                            drop_rate=Drop_rate,
                                            runs=Repeats,
                                            Model_num = m_count, 
                                            segment_length = segment_length)

    append_list_time1 = append_list_time1.append(res_list, ignore_index=True)
    tf.keras.backend.clear_session()
               

Sig Divide Time: 2.194899082183838
Sig Divide Time: 12.250142335891724
Sig Divide Time: 1.5449819564819336
Model Number 1
Time till start of create model 6.0821010265499353e-05
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[2048,2720,16] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/Model/residual_block_5/conv1D_1/Pad (defined at notebooks/CWRU_TCN/tcn_ed.py:157) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[div_no_nan/ReadVariableOp_1/_178]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[2048,2720,16] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/Model/residual_block_5/conv1D_1/Pad (defined at notebooks/CWRU_TCN/tcn_ed.py:157) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add r

In [6]:
tf.keras.backend.clear_session()
b_size = 1024 #normally 2048 but 512 on colab
b_size2 = int(b_size / 4)
step_size = 300
step_length=step_size
segment_length = 2400
with mirrored_strategy.scope():
    train_dataset, test_dataset, y_s1 = Create_Batch(data_path, split_perc, segment_length, step_length, b_size)
    val_dataset, X_v1, predict_labels= create_pred_batch(Val_path=Val_path, segment_length=segment_length, step_length=step_size, b_size=int(b_size/4))
waste_split = 0.1
with mirrored_strategy.scope():
    
    #Tuned Param dropout
    Learning_Rate = 0.00125
    Drop_rate = 0.175
    Repeats = 75

    m_count = 1
    print('Model Number %s' % m_count)
    history, y_preds, res_list = train_func2(train_data=train_dataset,
                                            test_dataset=test_dataset,
                                            Val_dataset=val_dataset,
                                            predict_labels=predict_labels, 
                                            filt_num=16,
                                            kernel_num=11,
                                            dilation=7,
                                            stack=1,
                                            learn_r=Learning_Rate,
                                            drop_rate=Drop_rate,
                                            runs=Repeats,
                                            Model_num = m_count, 
                                            segment_length = segment_length)
    append_list_time1 = append_list_time1.append(res_list, ignore_index=True)
    Learning_Rate = 0.0125
    Drop_rate = 0.3
    Repeats = 75

    m_count = 2
    print('Model Number %s' % m_count)
    history, y_preds, res_list = train_func2(train_data=train_dataset,
                                            test_dataset=test_dataset,
                                            Val_dataset=val_dataset,
                                            predict_labels=predict_labels, 
                                            filt_num=16,
                                            kernel_num=11,
                                            dilation=7,
                                            stack=1,
                                            learn_r=Learning_Rate,
                                            drop_rate=Drop_rate,
                                            runs=Repeats,
                                            Model_num = m_count, 
                                            segment_length = segment_length)

    append_list_time1 = append_list_time1.append(res_list, ignore_index=True)
    tf.keras.backend.clear_session()
               

Sig Divide Time: 1.670323371887207
Sig Divide Time: 7.745492219924927
Sig Divide Time: 1.1999897956848145
Model Number 1
Time till start of create model 6.114200004958548e-05
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/de

KeyboardInterrupt: 

In [ ]:
tf.keras.backend.clear_session()
b_size = 512 #normally 2048 but 512 on colab
b_size2 = int(b_size / 4)
step_size = 300
step_length=step_size
segment_length = 2400
with mirrored_strategy.scope():
    train_dataset, test_dataset, y_s1 = Create_Batch(data_path, split_perc, segment_length, step_length, b_size)
    val_dataset, X_v1, predict_labels= create_pred_batch(Val_path=Val_path, segment_length=segment_length, step_length=step_size, b_size=int(b_size/4))
waste_split = 0.1
with mirrored_strategy.scope():
    
    #Tuned Param dropout
    Learning_Rate = 0.0125
    Drop_rate = 0.175
    Repeats = 75

    m_count = 1
    print('Model Number %s' % m_count)
    history, y_preds, res_list = train_func2(train_data=train_dataset,
                                            test_dataset=test_dataset,
                                            Val_dataset=val_dataset,
                                            predict_labels=predict_labels, 
                                            filt_num=16,
                                            kernel_num=11,
                                            dilation=7,
                                            stack=1,
                                            learn_r=Learning_Rate,
                                            drop_rate=Drop_rate,
                                            runs=Repeats,
                                            Model_num = m_count, 
                                            segment_length = segment_length)
    append_list_time1 = append_list_time1.append(res_list, ignore_index=True)
    Learning_Rate = 0.0125
    Drop_rate = 0.3
    Repeats = 75

    m_count = 2
    print('Model Number %s' % m_count)
    history, y_preds, res_list = train_func2(train_data=train_dataset,
                                            test_dataset=test_dataset,
                                            Val_dataset=val_dataset,
                                            predict_labels=predict_labels, 
                                            filt_num=16,
                                            kernel_num=11,
                                            dilation=7,
                                            stack=1,
                                            learn_r=Learning_Rate,
                                            drop_rate=Drop_rate,
                                            runs=Repeats,
                                            Model_num = m_count, 
                                            segment_length = segment_length)

    append_list_time1 = append_list_time1.append(res_list, ignore_index=True)
    tf.keras.backend.clear_session()
               

In [ ]:
tf.keras.backend.clear_session()
b_size = 256 #normally 2048 but 512 on colab
b_size2 = int(b_size / 4)
step_size = 300
step_length=step_size
segment_length = 2400
with mirrored_strategy.scope():
    train_dataset, test_dataset, y_s1 = Create_Batch(data_path, split_perc, segment_length, step_length, b_size)
    val_dataset, X_v1, predict_labels= create_pred_batch(Val_path=Val_path, segment_length=segment_length, step_length=step_size, b_size=int(b_size/4))
waste_split = 0.1
with mirrored_strategy.scope():
    
    #Tuned Param dropout
    Learning_Rate = 0.0125
    Drop_rate = 0.175
    Repeats = 75

    m_count = 1
    append_list_time1 = pd.DataFrame
    print('Model Number %s' % m_count)
    history, y_preds, res_list = train_func2(train_data=train_dataset,
                                            test_dataset=test_dataset,
                                            Val_dataset=val_dataset,
                                            predict_labels=predict_labels, 
                                            filt_num=16,
                                            kernel_num=11,
                                            dilation=7,
                                            stack=1,
                                            learn_r=Learning_Rate,
                                            drop_rate=Drop_rate,
                                            runs=Repeats,
                                            Model_num = m_count, 
                                            segment_length = segment_length)
    
    Learning_Rate = 0.0125
    Drop_rate = 0.3
    Repeats = 75
    append_list_time1 = append_list_time1.append(res_list, ignore_index=True)
    m_count = 2
    append_list_time1 = pd.DataFrame
    print('Model Number %s' % m_count)
    history, y_preds, res_list = train_func2(train_data=train_dataset,
                                            test_dataset=test_dataset,
                                            Val_dataset=val_dataset,
                                            predict_labels=predict_labels, 
                                            filt_num=16,
                                            kernel_num=11,
                                            dilation=7,
                                            stack=1,
                                            learn_r=Learning_Rate,
                                            drop_rate=Drop_rate,
                                            runs=Repeats,
                                            Model_num = m_count, 
                                            segment_length = segment_length)

    append_list_time1 = append_list_time1.append(res_list, ignore_index=True)
    tf.keras.backend.clear_session()
               